In [3]:
from keras import layers
import tensorflow as tf
import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)


df = pd.read_csv('../csv_dateien/starcraftFinalcsvs/allRunsButOnlyTerran.csv')

conditions = [
    (df['fraction'] == "Protoss"), #1
    (df['fraction'] == "Terraner"), #2
    (df['fraction'] == "Zerg"), #3
    (df['fraction'] == "T"), #2
]

values = [1, 2, 3, 2]

df["fraction"] = np.select(conditions, values)
df=df[df["total_army"]>0]
df=df[df["fraction"]==2]
df=df[df["supply"]<=200]
df["ratio"] = df["total_workers"]/df["total_army"]
df = df[df["ratio"] < 4]

dfPrepSample = df.drop(columns=["player","winner","replay_filename", "Unnamed: 0", "total_workers", "total_army_value","fraction"])
len(dfPrepSample)
dfPrepSample

,resource_mining,total_army,supply,time,ratio
1,28154,96.0,174.0,836,0.479167
6,40998,120.0,196.0,934,0.550000
15,13965,45.0,117.0,583,1.200000
19,9595,26.0,62.0,449,1.346154
24,38030,98.0,165.0,1102,0.530612
...,...,...,...,...,...
5888,7364,17.0,69.0,381,2.411765
5889,25845,80.0,173.0,878,0.762500
5893,26351,70.0,157.0,795,1.000000
5894,24263,76.0,165.0,754,0.921053


In [4]:
df_sample = dfPrepSample.sample(frac=1, random_state=12)
dfTrainSample = df_sample.sample(frac=0.8)

dfTrainSample.head()

,resource_mining,total_army,supply,time,ratio
3227,45808,82.0,181.0,892,0.743902
3533,11764,21.0,93.0,497,2.380952
3863,28230,57.0,133.0,759,1.228070
4545,48687,80.0,141.0,1091,0.562500
2773,5702,13.0,46.0,334,2.076923


In [5]:
dfTestSample = df_sample.drop(dfTrainSample.index)

dfTestSample.head()

,resource_mining,total_army,supply,time,ratio
2596,14059,42.0,102.0,605,1.119048
4734,43239,103.0,188.0,1059,0.718447
850,8379,24.0,63.0,395,1.166667
5537,7246,13.0,54.0,374,2.769231
5835,27899,59.0,141.0,709,1.254237


In [6]:
dfTrainFeatures = dfTrainSample.copy()
dfTrainFeatures

,resource_mining,total_army,supply,time,ratio
3227,45808,82.0,181.0,892,0.743902
3533,11764,21.0,93.0,497,2.380952
3863,28230,57.0,133.0,759,1.228070
4545,48687,80.0,141.0,1091,0.562500
2773,5702,13.0,46.0,334,2.076923
...,...,...,...,...,...
5307,10576,31.0,70.0,480,1.322581
5673,20904,24.0,85.0,696,1.833333
4256,10559,34.0,94.0,490,1.352941
2904,20867,41.0,133.0,680,1.536585


In [7]:

dfTrainLabels = dfTrainFeatures.pop("total_army")
dfTrainFeatures = np.array(dfTrainFeatures)
dfTrainFeatures

array([[45808.   ,   181.   ,   892.   ,     0.744],
       [11764.   ,    93.   ,   497.   ,     2.381],
       [28230.   ,   133.   ,   759.   ,     1.228],
       ...,
       [10559.   ,    94.   ,   490.   ,     1.353],
       [20867.   ,   133.   ,   680.   ,     1.537],
       [ 6363.   ,    39.   ,   296.   ,     2.556]])

In [8]:
# normalize = layers.Normalization()
# normalize.adapt(dfTrainFeatures)

In [9]:
sc_model = tf.keras.Sequential([layers.Dense(3, activation="relu"),layers.Dense(3, activation="relu"),layers.Dense(3, activation="relu"), layers.Dense(1)]) # letzte lösung 30/5

sc_model.compile(loss = tf.keras.losses.MeanSquaredError(),
                      optimizer = tf.keras.optimizers.Adam())

In [10]:
sc_model.fit(dfTrainFeatures, dfTrainLabels, epochs=1000)  #350

Epoch 1/1000
70/70 [==============================] - 1s 934us/step - loss: 2751.5027
Epoch 2/1000
70/70 [==============================] - 0s 694us/step - loss: 2745.3171
Epoch 3/1000
70/70 [==============================] - 0s 668us/step - loss: 2739.1431
Epoch 4/1000
70/70 [==============================] - 0s 656us/step - loss: 2733.0032
Epoch 5/1000
70/70 [==============================] - 0s 633us/step - loss: 2726.8816
Epoch 6/1000
70/70 [==============================] - 0s 614us/step - loss: 2720.7874
Epoch 7/1000
70/70 [==============================] - 0s 624us/step - loss: 2714.6702
Epoch 8/1000
70/70 [==============================] - 0s 603us/step - loss: 2708.5886
Epoch 9/1000
70/70 [==============================] - 0s 605us/step - loss: 2702.5188
Epoch 10/1000
70/70 [==============================] - 0s 609us/step - loss: 2696.4556
Epoch 11/1000
70/70 [==============================] - 0s 650us/step - loss: 2690.4084
Epoch 12/1000
70/70 [==============================]

In [11]:
sc_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 3)                 15        
                                                                 
 dense_1 (Dense)             (None, 3)                 12        
                                                                 
 dense_2 (Dense)             (None, 3)                 12        
                                                                 
 dense_3 (Dense)             (None, 1)                 4         
                                                                 
Total params: 43
Trainable params: 43
Non-trainable params: 0
_________________________________________________________________


In [12]:
dfTestFeatures = dfTestSample.copy()

dfTestLabels = dfTestFeatures.pop("total_army")
dfTestFeatures = np.array(dfTestFeatures)
dfTestLabels.head()

2596     42.0
4734    103.0
850      24.0
5537     13.0
5835     59.0
Name: total_army, dtype: float64

In [13]:
from sklearn.metrics import r2_score


predictedValues = sc_model.predict(dfTestFeatures)
r2_score_sc = (r2_score(dfTestLabels, predictedValues))
stri = ""
if(r2_score_sc<0):
    stri = "negative" + str(r2_score_sc)
else:
    stri = str(r2_score_sc)

print(f"The R^2 value in the test set is {stri}")

18/18 [==============================] - 0s 561us/step
The R^2 value in the test set is negative-0.0012786266275115477


In [14]:
sc_model.save('sc_ts_model.h5')
